<a href="https://colab.research.google.com/github/essiesalari/Australian-Celebrities-Face-Recognition-with-Transfer-Learning/blob/main/Australian_Celebrities_Face_Recognition_VGG16.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Australian Celebrities Face Recognition-VGG16**